In [1]:
# Import libraries

import pandas as pd

In [3]:
# Read data

data = pd.read_excel('data/accuration_test2.xlsx')
data['pattern_webb'] = data['pattern_webb'].apply(lambda x:list(eval(str(x))))
data['pattern_raharjana'] = data['pattern_raharjana'].apply(lambda x:list(eval(str(x))))
data['pattern_johann'] = data['pattern_johann'].apply(lambda x:list(eval(str(x))))
data['expected_result'] = data['expected_result'].apply(lambda x:list(eval(str(x))))

data.head()

,title_and_abstract,pattern_webb,pattern_johann,pattern_raharjana,expected_result
0,fire detection system based on artificial inte...,"[(detect, fire), (generate, video), (analyze, ...","[(base, on, artificial, intelligence), (base, ...","[(base, on, ai), (base, on, ai), (detect, fire...","[(detect, fire), (generate, video, image), (ex..."
1,the present disclosure relates to a fire detec...,"[(detect, fire), (generate, video), (analyze, ...","[(detect, fire), (use, deep, learning), (inclu...","[(base, on, ai), (base, on, ai), (detect, fire...","[(detect, fire), (generate, video, image), (ex..."
2,embedding artificial intelligence for balancin...,"[(balance, access), (process, load), (enable, ...","[(embed, artificial, intelligence), (balance, ...","[(balance, access, point), (process, load), (e...","[(balance, access, point), (determine, rssi, v..."
3,responsive to a cpu load of a specific access ...,"[(threshold, value), (determine, rssi)]","[(threshold, value), (determine, rssi)]","[(threshold, value), (determine, rssi, value),...","[(determine, rssi, value)]"
4,optimization techniques for artificial intelli...,"[(receive, annotation), (determine, document),...","[(receive, annotation), (annotate, first, set)...","[(comprise, select, from, a, pool), (receive, ...","[(generate, a, natural, language, model), (gen..."


In [9]:
# find the right predicted chunk
def find_true(index, column):
    common_tuples = []
    for tuple1 in column[index]:
        for tuple2 in data['expected_result'][index]:
            if any(word in tuple2 for word in tuple1):
                common_tuples.append(tuple2)
                break
    return common_tuples

data['true_webb'] = data.apply(lambda row: find_true(row.name, data['pattern_webb']), axis=1)
data['true_raharjana'] = data.apply(lambda row: find_true(row.name, data['pattern_raharjana']), axis=1)
data['true_johann'] = data.apply(lambda row: find_true(row.name, data['pattern_johann']), axis=1)
data['true_webb']

0           [(detect, fire), (generate, video, image)]
1           [(detect, fire), (generate, video, image)]
2    [(balance, access, point), (determine, rssi, v...
3    [(determine, rssi, value), (determine, rssi, v...
4    [(receive, annotation), (select, from, a, pool...
5    [(receive, annotation), (select, from, a, pool...
6    [(create, a, secure, distributed, data, valida...
7    [(validate, paper, forms), (select, a, first, ...
8           [(personalize, dialogue), (perform, card)]
9    [(optimize, planning), (select, an, optimal, m...
Name: true_webb, dtype: object

In [11]:
# find the false predicted chunk from verb-noun pattern
def find_false(index, column):
    disjoint_tuples = []
    for tuple1 in column[index]:
        found_common = False
        for tuple2 in data['expected_result'][index]:
            if any(word in tuple2 for word in tuple1):
                found_common = True
                break
        if not found_common:
            disjoint_tuples.append(tuple1)
    return disjoint_tuples

data['false_webb'] = data.apply(lambda row: find_false(row.name, data['pattern_webb']), axis=1)
data['false_johann'] = data.apply(lambda row: find_false(row.name, data['pattern_johann']), axis=1)
data['false_raharjana'] = data.apply(lambda row: find_false(row.name, data['pattern_raharjana']), axis=1)
data['false_webb']

0                       [(analyze, part)]
1                       [(analyze, part)]
2    [(process, load), (enable, network)]
3                                      []
4                                      []
5                                      []
6                                      []
7                                      []
8                                      []
9                 [(include, management)]
Name: false_webb, dtype: object

In [13]:
# find the unpredicted chunk from verb-noun pattern

def find_unpredicted(index, column_true):
    result = [i for i in data['expected_result'][index] if i not in column_true[index]]
    return result

data['unpredicted_webb'] = data.apply(lambda row: find_unpredicted(row.name, data['true_webb']), axis=1)
data['unpredicted_raharjana'] = data.apply(lambda row: find_unpredicted(row.name, data['true_raharjana']), axis=1)
data['unpredicted_johann'] = data.apply(lambda row: find_unpredicted(row.name, data['true_johann']), axis=1)
data['unpredicted_webb']

0    [(extract, a, motion, object), (generate, a, f...
1    [(extract, a, motion, object), (generate, a, f...
2                                                   []
3                                                   []
4    [(generate, a, natural, language, model), (gen...
5    [(generate, a, natural, language, model), (gen...
6    [(receive, paper, form, data), (determine, fir...
7    [(receive, paper, form, data), (determine, fir...
8    [(track, player, attributes), (track, game, st...
9    [(execute, an, order), (provide, an, optimal, ...
Name: unpredicted_webb, dtype: object

In [14]:
# count percentage of pattern Webb accuracy

tp_webb = data['true_webb'].apply(len).sum()
fp_webb = data['false_webb'].apply(len).sum()
fn_webb = data['unpredicted_webb'].apply(len).sum()
precision_webb = tp_webb / (tp_webb + fp_webb)
recall_webb = tp_webb / (tp_webb + fn_webb)
f1score_webb = 2 * (precision_webb * recall_webb) / (precision_webb + recall_webb)

print("Hasil chunk pattern Webb\n")
print("True positive = ", tp_webb)
print("False positive = ", fp_webb)
print("False negative = ", fn_webb)
print("\nPrecision = ", precision_webb)
print("Recall = ", recall_webb)
print("F1 score = ", f1score_webb)


Hasil chunk pattern Webb

True positive =  27
False positive =  5
False negative =  51

Precision =  0.84375
Recall =  0.34615384615384615
F1 score =  0.4909090909090909


In [15]:
# count percentage of pattern Johann accuracy

tp_johann = data['true_johann'].apply(len).sum()
fp_johann = data['false_johann'].apply(len).sum()
fn_johann = data['unpredicted_johann'].apply(len).sum()
precision_johann = tp_johann / (tp_johann + fp_johann)
recall_johann = tp_johann / (tp_johann + fn_johann)
f1score_johann = 2 * (precision_johann * recall_johann) / (precision_johann + recall_johann)

print("Hasil chunk pattern Johann\n")
print("True positive = ", tp_johann)
print("False positive = ", fp_johann)
print("False negative = ", fn_johann)
print("\nPrecision = ", precision_johann)
print("Recall = ", recall_johann)
print("F1 score = ", f1score_johann)

Hasil chunk pattern Johann

True positive =  50
False positive =  13
False negative =  39

Precision =  0.7936507936507936
Recall =  0.5617977528089888
F1 score =  0.6578947368421053


In [16]:
# count percentage of pattern Raharjana accuracy

tp_raharjana = data['true_raharjana'].apply(len).sum()
fp_raharjana = data['false_raharjana'].apply(len).sum()
fn_raharjana = data['unpredicted_raharjana'].apply(len).sum()
precision_raharjana = tp_raharjana / (tp_raharjana + fp_raharjana)
recall_raharjana = tp_raharjana / (tp_raharjana + fn_raharjana)
f1score_raharjana = 2 * (precision_raharjana * recall_raharjana) / (precision_raharjana + recall_raharjana)

print("Hasil chunk pattern Raharjana\n")
print("True positive = ", tp_raharjana)
print("False positive = ", fp_raharjana)
print("False negative = ", fn_raharjana)
print("\nPrecision = ", precision_raharjana)
print("Recall = ", recall_raharjana)
print("F1 score = ", f1score_raharjana)

Hasil chunk pattern Raharjana

True positive =  59
False positive =  16
False negative =  34

Precision =  0.7866666666666666
Recall =  0.6344086021505376
F1 score =  0.7023809523809524


In [22]:
# hasile

data.to_excel('export_result/accuration_test_result2.xlsx', index=False)